In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import os
import cv2
from keras.layers import Dense, Flatten
import torch

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Define the paths to your image folders
train_path = "C:/Users/hp/Documents/Mtech/GurNum/GurNum"
val_path = "C:/Users/hp/Documents/Mtech/GurNum/GurNum"

In [ ]:
# Set the path to the folder containing the 'train' folder
data_dir = train_path
# Set the image size
img_size = (32, 32)
# Create empty lists for the images and labels
images = []
labels = []
# Loop over each folder from '0' to '9'
for label in range(10):
    folder_path = os.path.join(data_dir, 'train', str(label))
     # Loop over each image in the folder
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if file_path.endswith(('.tiff','.bmp')):
             # Load the image and resize it to the desired size
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, img_size)
            # Append the image and label to the lists
            images.append(img)
            labels.append(label)
            
# Convert the lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)
# Save the arrays in NumPy format
np.save('x_train.npy', images)
np.save('y_train.npy', labels)


In [ ]:
# Set the path to the folder containing the 'val' folder
data_dir_val = val_path
# Set the image size
img_size_val = (32, 32)
# Create empty lists for the images and labels
images_val = []
labels_val = []
# Loop over each folder from '0' to '9'
for label in range(10):
    folder_path = os.path.join(data_dir_val, 'val\\', str(label))

    # Loop over each image in the folder
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if file_path.endswith(('.tiff','.bmp')):
            # Load the image and resize it to the desired size
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, img_size_val)
            # Append the image and label to the lists
            images_val.append(img)
            labels_val.append(label)
# Convert the lists to NumPy arrays
images_val = np.array(images_val)
labels_val = np.array(labels_val)
# Save the arrays in NumPy format
np.save('x_test.npy', images_val)
np.save('y_test.npy', labels_val)

In [ ]:
# Load the dataset
x1_train = np.load('x_train.npy')
y1_train = np.load('y_train.npy')
x1_test = np.load('x_test.npy')
y1_test = np.load('y_test.npy')

In [ ]:
# test the images are loaded correctly
print(len(x1_train))
print(len(x1_test))
x1_train[0].shape
x1_train[0]
plt.matshow(x1_train[0])
plt.matshow(x1_train[999])
print(x1_train.shape)
print(x1_test.shape)
y1_train
y1_test
plt.matshow(x1_test[150])

In [ ]:
model = keras.Sequential([keras.layers.Flatten(),
    keras.layers.Dense(10, input_shape=(1024,),activation = 'sigmoid')
])
# compile the nn
model.compile(optimizer='adam',
 loss='sparse_categorical_crossentropy',
 metrics=['accuracy']
 )
# train the model
# some 10 iterations done here
model.fit(x1_train, y1_train,epochs= 10, validation_data=(x1_test, y1_test))

In [ ]:
x1_train_scaled = x1_train/255
x1_test_scaled = x1_test/255
model.fit(x1_train_scaled, y1_train,epochs= 10, validation_data=(x1_test_scaled, y1_test))

In [ ]:
model.evaluate(x_test_scaled,y_test)


In [ ]:
# predict 1st image
plt.matshow(x_test[0])
y1_predicted = model.predict(x_test_scaled)
y1_predicted[0]
# this showing the 10 results for the input '0', we need to look for the value which is max
print('Predicted Value is ',np.argmax(y1_predicted[0]))
# test some more values
plt.matshow(x1_test[88])
print('Predicted Value is ',np.argmax(y1_predicted[88]))
plt.matshow(x1_test[177])
print('Predicted Value is ',np.argmax(y1_predicted[177]))

In [ ]:
# convert to concrete values
y1_predicted_labels=[np.argmax(i) for i in y_predicted]
print(y1_predicted_labels, len(y1_predicted_labels))
conf_mat = tf.math.confusion_matrix(labels=y1_test, predictions=y1_predicted_labels)
conf_mat

In [ ]:
import seaborn as sn
plt.figure(figsize = (10,10))
sn.heatmap(conf_mat,annot=True,fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Actual')

In [ ]:
model2 = keras.Sequential([
 keras.layers.Flatten(),
 keras.layers.Dense(1024,input_shape=(1024,), activation='relu'),
 keras.layers.Dense(10, activation='softmax')
])
# compile the nn
model2.compile(optimizer='adam',
 loss='sparse_categorical_crossentropy',
 metrics=['accuracy']
 )
# train the model
# some 10 iterations done here
history = model2.fit(x1_train_scaled, y1_train,epochs= 10, validation_data=(x1_test_scaled, y1_test))

In [ ]:
# evaluate test dataset on modified model
model2.evaluate(x1_test_scaled,y1_test)

In [ ]:
# build confusion matrix to see how our prediction looks like
# convert to concrete values
y1_predicted = model2.predict(x1_test_scaled)
y1_predicted[0]
y1_predicted_labels=[np.argmax(i) for i in y_predicted]
print(y1_predicted_labels, len(y1_predicted_labels))
conf_mat = tf.math.confusion_matrix(labels=y1_test, predictions=y1_predicted_labels)
conf_mat

In [ ]:
plt.figure(figsize = (10,10))
sn.heatmap(conf_mat,annot=True,fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Actual')

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x1_test, y1_test)
print('Test accuracy:', test_acc)
# Plot the training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()